In [2]:
import logging
import os
import threadpoolctl
import torch
from pathlib import Path
from lightning_fabric import seed_everything

import numpy as np
import pandas as pd
from rectools import Columns

from implicit.als import AlternatingLeastSquares

from rectools.dataset import Dataset
from rectools.metrics import MAP, calc_metrics, MeanInvUserFreq, Serendipity
from rectools.models import ImplicitALSWrapperModel
from rectools.models import SASRecModel
from rectools.models.nn.item_net import CatFeaturesItemNet, IdEmbeddingsItemNet

In [3]:
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

# For implicit ALS
os.environ["OPENBLAS_NUM_THREADS"] = "1"
threadpoolctl.threadpool_limits(1, "blas")

logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)
logger = logging.getLogger()

# Data

In [4]:
# %%time
# !wget -q https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip -O data_original.zip
# !unzip -o data_original.zip
# !rm data_original.zip

In [5]:
DATA_PATH = Path("data_original")

interactions = (
    pd.read_csv(DATA_PATH / 'interactions.csv', parse_dates=["last_watch_dt"])
    .rename(columns={"last_watch_dt": "datetime"})
)

# Split dataset

In [6]:
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)

# Split to train / test
max_date = interactions[Columns.Datetime].max()
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()
train.drop(train.query("total_dur < 300").index, inplace=True)

# drop items with less than 20 interactions in train
items = train["item_id"].value_counts()
items = items[items >= 20]
items = items.index.to_list()
train = train[train["item_id"].isin(items)]
    
# drop users with less than 2 interactions in train
users = train["user_id"].value_counts()
users = users[users >= 2]
users = users.index.to_list()
train = train[(train["user_id"].isin(users))]

users = train["user_id"].drop_duplicates().to_list()

# drop cold users from test
test_users_sasrec = test[Columns.User].unique()
cold_users = set(test[Columns.User]) - set(train[Columns.User])
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)
test_users = test[Columns.User].unique()


In [7]:
items = pd.read_csv(DATA_PATH / 'items.csv')

In [8]:
# Process item features to the form of a flatten dataframe
items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"
item_features = pd.concat((genre_feature, content_feature))

candidate_items = interactions['item_id'].drop_duplicates().astype(int)
test["user_id"] = test["user_id"].astype(int)
test["item_id"] = test["item_id"].astype(int)

catalog=train[Columns.Item].unique()

In [9]:
dataset_no_features = Dataset.construct(
    interactions_df=train,
)

dataset_item_features = Dataset.construct(
    interactions_df=train,
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

In [10]:
metrics_name = {
    'MAP': MAP,
    'MIUF': MeanInvUserFreq,
    'Serendipity': Serendipity
    

}
metrics = {}
for metric_name, metric in metrics_name.items():
    for k in (1, 5, 10):
        metrics[f'{metric_name}@{k}'] = metric(k=k)

# list with metrics results of all models
features_results = []

# SASRec

In [11]:
RANDOM_SEED = 32
torch.use_deterministic_algorithms(True)
seed_everything(RANDOM_SEED, workers=True)

Seed set to 32


32

## Softmax loss

In [13]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    item_net_block_types=(IdEmbeddingsItemNet, ),  # Use only item ids in ItemNetBlock
    recommend_device="cuda",
)


In [14]:
%%time
model.fit(dataset_no_features)

/data/home/amsemenov2/git/RecTools_origin/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params
---------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 927 K 
---------------------------------------------------------------
927 K     Trainable params
0         Non-trainable params
927 K     Total params
3.709     Total estimated model params size (MB)
/data/home/amsemenov2/git/RecTools_origin/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=143` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 5min 9s, sys: 6.07 s, total: 5min 15s
Wall time: 5min


In [15]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_no_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)

/data/home/amsemenov2/git/RecTools_origin/RecTools/examples/../rectools/models/nn/transformer_data_preparator.py:313: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/amsemenov2/git/RecTools_origin/RecTools/examples/../rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/data/home/amsemenov2/git/RecTools_origin/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a b

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 26.4 s, sys: 3.84 s, total: 30.2 s
Wall time: 21 s


In [16]:
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "softmax"
features_results.append(metric_values)


In [17]:
features_results

[{'MAP@1': 0.048488013974905805,
  'MAP@5': 0.0821411524595932,
  'MAP@10': 0.0912577746921091,
  'MIUF@1': 3.868676865340589,
  'MIUF@5': 4.506241791317061,
  'MIUF@10': 5.087812416018942,
  'Serendipity@1': 0.0010587683094952943,
  'Serendipity@5': 0.0008279085147448243,
  'Serendipity@10': 0.0007506236395264775,
  'model': 'softmax'}]

In [18]:
recos

,user_id,item_id,score,rank
0,73446,9728,2.073619,1
1,73446,7793,2.032206,2
2,73446,3784,1.862259,3
3,73446,12965,1.821924,4
4,73446,3182,1.760829,5
...,...,...,...,...
947045,857162,6809,2.055156,6
947046,857162,12995,1.811543,7
947047,857162,15362,1.524744,8
947048,857162,4495,1.470260,9


## BCE loss

In [20]:
RANDOM_SEED = 32
torch.use_deterministic_algorithms(True)
seed_everything(RANDOM_SEED, workers=True)

Seed set to 32


32

In [21]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    loss="BCE",
    n_negatives=2,
    item_net_block_types=(IdEmbeddingsItemNet, ),  # Use only item ids in ItemNetBlock
)


In [22]:
%%time
model.fit(dataset_no_features)

/data/home/amsemenov2/git/RecTools_origin/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params
---------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 927 K 
---------------------------------------------------------------
927 K     Trainable params
0         Non-trainable params
927 K     Total params
3.709     Total estimated model params size (MB)
/data/home/amsemenov2/git/RecTools_origin/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=143` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 5min 14s, sys: 7.46 s, total: 5min 21s
Wall time: 5min 10s


In [23]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_no_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn",
)

/data/home/amsemenov2/git/RecTools_origin/RecTools/examples/../rectools/models/nn/transformer_data_preparator.py:313: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/amsemenov2/git/RecTools_origin/RecTools/examples/../rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/data/home/amsemenov2/git/RecTools_origin/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a b

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 24.9 s, sys: 3.25 s, total: 28.1 s
Wall time: 20.1 s


In [24]:
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "bce"
features_results.append(metric_values)

In [25]:
recos

,user_id,item_id,score,rank
0,73446,9728,3.905239,1
1,73446,7829,3.675018,2
2,73446,7793,3.606547,3
3,73446,3784,3.505795,4
4,73446,6646,3.314492,5
...,...,...,...,...
947045,857162,3734,3.120592,6
947046,857162,12995,2.934546,7
947047,857162,12692,2.731019,8
947048,857162,657,2.596890,9


## gBCE loss

In [27]:
RANDOM_SEED = 32
torch.use_deterministic_algorithms(True)
seed_everything(RANDOM_SEED, workers=True)

Seed set to 32


32

In [28]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    loss="gBCE",
    n_negatives=256,
    gbce_t=0.75,
    item_net_block_types=(IdEmbeddingsItemNet, ),  # Use only item ids in ItemNetBlock
)

In [29]:
%%time
model.fit(dataset_no_features)

/data/home/amsemenov2/git/RecTools_origin/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params
---------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 927 K 
---------------------------------------------------------------
927 K     Trainable params
0         Non-trainable params
927 K     Total params
3.709     Total estimated model params size (MB)
/data/home/amsemenov2/git/RecTools_origin/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=143` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 2h 33min 27s, sys: 40.9 s, total: 2h 34min 8s
Wall time: 10min 45s


In [30]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_no_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)


/data/home/amsemenov2/git/RecTools_origin/RecTools/examples/../rectools/models/nn/transformer_data_preparator.py:313: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/amsemenov2/git/RecTools_origin/RecTools/examples/../rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/data/home/amsemenov2/git/RecTools_origin/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a b

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 25.2 s, sys: 3.15 s, total: 28.3 s
Wall time: 19.9 s


In [31]:
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "gBCE"
features_results.append(metric_values)

## Softmax with key_padding_mask + causal_mask

In [33]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    item_net_block_types=(IdEmbeddingsItemNet, ),  # Use only item ids in ItemNetBlock
    recommend_device="cuda",
    use_key_padding_mask=True,
)


In [34]:
%%time
model.fit(dataset_no_features)

/data/home/amsemenov2/git/RecTools_origin/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params
---------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 927 K 
---------------------------------------------------------------
927 K     Trainable params
0         Non-trainable params
927 K     Total params
3.709     Total estimated model params size (MB)
/data/home/amsemenov2/git/RecTools_origin/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=143` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 5min 17s, sys: 7.15 s, total: 5min 24s
Wall time: 5min 10s


In [35]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_no_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)

/data/home/amsemenov2/git/RecTools_origin/RecTools/examples/../rectools/models/nn/transformer_data_preparator.py:313: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/amsemenov2/git/RecTools_origin/RecTools/examples/../rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/data/home/amsemenov2/git/RecTools_origin/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a b

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 25.6 s, sys: 3.04 s, total: 28.6 s
Wall time: 19.8 s


In [36]:
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)

In [37]:
metric_values["model"] = "softmax_padding_mask"
features_results.append(metric_values)

In [38]:
features_df = (
    pd.DataFrame(features_results)
    .set_index("model")
    .sort_values(by=["MAP@10", "Serendipity@10"], ascending=False)
)
features_df

,MAP@1,MAP@5,MAP@10,MIUF@1,MIUF@5,MIUF@10,Serendipity@1,Serendipity@5,Serendipity@10
model,,,,,,,,,
softmax,0.048488,0.082141,0.091258,3.868677,4.506242,5.087812,0.001059,0.000828,0.000751
softmax_padding_mask,0.046903,0.080840,0.089952,4.044813,4.601506,5.091948,0.001064,0.000822,0.000720
gBCE,0.046173,0.080317,0.089186,3.186794,3.822623,4.538796,0.000640,0.000517,0.000505
bce,0.043094,0.073984,0.082944,3.598352,4.304013,4.916007,0.000477,0.000486,0.000502


### sasrec with item ids embeddings in ItemNetBlock

In [40]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    item_net_block_types=(IdEmbeddingsItemNet, ),  # Use only item ids in ItemNetBlock
)

In [41]:
%%time
model.fit(dataset_no_features)

/data/home/amsemenov2/git/RecTools_origin/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params
---------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 927 K 
---------------------------------------------------------------
927 K     Trainable params
0         Non-trainable params
927 K     Total params
3.709     Total estimated model params size (MB)
/data/home/amsemenov2/git/RecTools_origin/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=143` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 5min 21s, sys: 8.59 s, total: 5min 29s
Wall time: 5min 14s


In [42]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_no_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)

/data/home/amsemenov2/git/RecTools_origin/RecTools/examples/../rectools/models/nn/transformer_data_preparator.py:313: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/amsemenov2/git/RecTools_origin/RecTools/examples/../rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/data/home/amsemenov2/git/RecTools_origin/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a b

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 27.6 s, sys: 3.45 s, total: 31.1 s
Wall time: 22.3 s


In [43]:
# TODO: drop `apply(str)`
recos["item_id"] = recos["item_id"].apply(str)
test["item_id"] = test["item_id"].astype(str)
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "sasrec_ids"
features_results.append(metric_values)

### sasrec with item ids and category features embeddings in ItemNetBlock

In [44]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    item_net_block_types=(IdEmbeddingsItemNet, CatFeaturesItemNet),  # Use item ids and cat features in ItemNetBlock
)

In [45]:
#%%time
model.fit(dataset_item_features)

/data/home/amsemenov2/git/RecTools_origin/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params
---------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 935 K 
---------------------------------------------------------------
935 K     Trainable params
0         Non-trainable params
935 K     Total params
3.742     Total estimated model params size (MB)
/data/home/amsemenov2/git/RecTools_origin/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=143` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

/data/home/amsemenov2/git/RecTools_origin/RecTools/examples/../rectools/dataset/features.py:424: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")


`Trainer.fit` stopped: `max_epochs=5` reached.


In [46]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_item_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)

/data/home/amsemenov2/git/RecTools_origin/RecTools/examples/../rectools/models/nn/transformer_data_preparator.py:313: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/amsemenov2/git/RecTools_origin/RecTools/examples/../rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/data/home/amsemenov2/git/RecTools_origin/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a b

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 28.9 s, sys: 3.6 s, total: 32.5 s
Wall time: 22.2 s


In [47]:
# TODO: drop `apply(str)`
recos["item_id"] = recos["item_id"].apply(str)
test["item_id"] = test["item_id"].astype(str)
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "sasrec_ids_cat"
features_results.append(metric_values)

### sasrec with category item features embeddings in ItemNetBlock

In [49]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    item_net_block_types=(CatFeaturesItemNet, ),  # Use only cat item features in ItemNetBlock
)

In [50]:
#%%time
model.fit(dataset_item_features)

/data/home/amsemenov2/git/RecTools_origin/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params
---------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 211 K 
---------------------------------------------------------------
211 K     Trainable params
0         Non-trainable params
211 K     Total params
0.847     Total estimated model params size (MB)
/data/home/amsemenov2/git/RecTools_origin/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=143` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

/data/home/amsemenov2/git/RecTools_origin/RecTools/examples/../rectools/dataset/features.py:424: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")
`Trainer.fit` stopped: `max_epochs=5` reached.


In [51]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_item_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)

/data/home/amsemenov2/git/RecTools_origin/RecTools/examples/../rectools/models/nn/transformer_data_preparator.py:313: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/amsemenov2/git/RecTools_origin/RecTools/examples/../rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/data/home/amsemenov2/git/RecTools_origin/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a b

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 24.3 s, sys: 2.53 s, total: 26.9 s
Wall time: 19.2 s


In [52]:
# TODO: drop `apply(str)`
recos["item_id"] = recos["item_id"].apply(str)
test["item_id"] = test["item_id"].astype(str)
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "sasrec_cat"
features_results.append(metric_values)

In [53]:
features_results

[{'MAP@1': 0.048488013974905805,
  'MAP@5': 0.0821411524595932,
  'MAP@10': 0.0912577746921091,
  'MIUF@1': 3.868676865340589,
  'MIUF@5': 4.506241791317061,
  'MIUF@10': 5.087812416018942,
  'Serendipity@1': 0.0010587683094952943,
  'Serendipity@5': 0.0008279085147448243,
  'Serendipity@10': 0.0007506236395264775,
  'model': 'softmax'},
 {'MAP@1': 0.043093656973408605,
  'MAP@5': 0.07398385923675324,
  'MAP@10': 0.08294397368660968,
  'MIUF@1': 3.598352040478207,
  'MIUF@5': 4.304012747602231,
  'MIUF@10': 4.916007085918255,
  'Serendipity@1': 0.0004767361940337927,
  'Serendipity@5': 0.0004863863900258476,
  'Serendipity@10': 0.000501558401864858,
  'model': 'bce'},
 {'MAP@1': 0.046172701078265196,
  'MAP@5': 0.08031691396493008,
  'MAP@10': 0.08918565294041045,
  'MIUF@1': 3.1867943881001892,
  'MIUF@5': 3.822622559312681,
  'MIUF@10': 4.538795926216838,
  'Serendipity@1': 0.0006401436142825035,
  'Serendipity@5': 0.0005169224313850365,
  'Serendipity@10': 0.0005046814909982423,
  '

### Item to item

In [54]:
target_items = [13865, 4457, 15297]

In [55]:
%%time
recos = model.recommend_to_items(
    target_items=target_items, 
    dataset=dataset_no_features,
    k=10,
    filter_itself=True,
    items_to_recommend=None, #white_list,
)

CPU times: user 1.43 s, sys: 197 ms, total: 1.63 s
Wall time: 1.07 s


In [56]:
recos

,target_item_id,item_id,score,rank
0,13865,15648,1.000000,1
1,13865,3386,1.000000,2
2,13865,16194,0.908769,3
3,13865,147,0.908769,4
4,13865,12586,0.908769,5
5,13865,12309,0.908769,6
6,13865,6661,0.908769,7
7,13865,2255,0.908769,8
8,13865,4130,0.908769,9
9,13865,9109,0.908769,10


In [57]:
# TODO: change model for recos (here is the last one trained and is is the worst in quality)
recos.merge(items[["item_id", "title"]], on="item_id")

,target_item_id,item_id,score,rank,title
0,13865,15648,1.000000,1,Черное золото
1,13865,3386,1.000000,2,Спартак
2,13865,16194,0.908769,3,Голубая линия
3,13865,147,0.908769,4,Единичка
4,13865,12586,0.908769,5,Вспоминая 1942
5,13865,12309,0.908769,6,Враг у ворот
6,13865,6661,0.908769,7,Солдатик
7,13865,2255,0.908769,8,Пленный
8,13865,4130,0.908769,9,Пустота
9,13865,9109,0.908769,10,Последняя битва
